In [1]:
import pandas as pd
import re
import pymorphy2
import joblib as pickle

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from sklearn import tree

In [3]:
df = pd.read_csv('lenta-ru-news.csv')

C:\Users\liz_o\AppData\Local\Temp\ipykernel_11048\3394781730.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('lenta-ru-news.csv')


In [4]:
df.head()

,url,title,text,topic,tags,date
0,https://lenta.ru/news/1914/09/16/hungarnn/,1914. Русские войска вступили в пределы Венгрии,Бои у Сопоцкина и Друскеник закончились отступ...,Библиотека,Первая мировая,1914/09/16
1,https://lenta.ru/news/1914/09/16/lermontov/,1914. Празднование столетия М.Ю. Лермонтова от...,"Министерство народного просвещения, в виду про...",Библиотека,Первая мировая,1914/09/16
2,https://lenta.ru/news/1914/09/17/nesteroff/,1914. Das ist Nesteroff!,"Штабс-капитан П. Н. Нестеров на днях, увидев в...",Библиотека,Первая мировая,1914/09/17
3,https://lenta.ru/news/1914/09/17/bulldogn/,1914. Бульдог-гонец под Льежем,Фотограф-корреспондент Daily Mirror рассказыва...,Библиотека,Первая мировая,1914/09/17
4,https://lenta.ru/news/1914/09/18/zver/,1914. Под Люблином пойман швабский зверь,"Лица, приехавшие в Варшаву из Люблина, передаю...",Библиотека,Первая мировая,1914/09/18


In [5]:
df.shape

(800975, 6)

In [6]:
df.tags.value_counts()

Все               453762
Политика           40716
Общество           35202
Украина            22523
Происшествия       19825
                   ...  
Мировой опыт           6
Нацпроекты             6
Вооружение             3
Инновации              1
69-я параллель         1
Name: tags, Length: 94, dtype: int64

In [7]:
df = df[df.topic.isin(['Россия','Мир','Экономика','Спорт','Культура'])].copy()

In [8]:
df.shape

(494804, 6)

In [9]:
df.head()

,url,title,text,topic,tags,date
5,https://lenta.ru/news/1999/08/31/stancia_mir/,"Космонавты сомневаются в надежности ""Мира""",Как стало известно агентству Ассошиэйтед Пресс...,Россия,Все,1999/08/31
6,https://lenta.ru/news/1999/08/31/vzriv/,Взрыв в центре Москвы: пострадало 30 человек,В зале игровых автоматов в третьем ярусе подзе...,Россия,Все,1999/08/31
7,https://lenta.ru/news/1999/08/31/credit_japs/,Япония кредитует Россию на полтора миллиарда д...,Япония приняла решение разморозить кредиты Рос...,Россия,Все,1999/08/31
8,https://lenta.ru/news/1999/08/31/diana/,Британцы отмечают двухлетие смерти Дианы,Британцы отмечают сегодня скорбную дату - втор...,Мир,Все,1999/08/31
9,https://lenta.ru/news/1999/08/31/mvf/,Отмытые через Bank of NY деньги не имели отнош...,В понедельник директор департамента внешних св...,Россия,Все,1999/08/31


In [10]:
df.isna().sum()

url          0
title        0
text         5
topic        0
tags     15328
date         0
dtype: int64

In [11]:
df[df.text.isna()]['title'].values[0]

'Прокуратура не\xa0нашла национальной почвы в\xa0нападении на\xa0индийского студента'

In [12]:
df.dropna(subset=['text'],inplace = True)

In [13]:
df.topic.value_counts()

Россия       160442
Мир          136620
Экономика     79528
Спорт         64413
Культура      53796
Name: topic, dtype: int64

In [14]:
df.topic.value_counts(normalize=True)

Россия       0.324257
Мир          0.276112
Экономика    0.160728
Спорт        0.130180
Культура     0.108723
Name: topic, dtype: float64

In [15]:
df.isna().sum()

url          0
title        0
text         0
topic        0
tags     15328
date         0
dtype: int64

In [16]:
df[df.text.isna()]['title'].values

array([], dtype=object)

In [17]:
df.shape

(494799, 6)

In [18]:
RANDOM_STATE = 42

### Базовая обработка

In [19]:
text_transformer = TfidfVectorizer()

In [20]:
%%time
text = text_transformer.fit_transform(df['text'])

CPU times: total: 1min 38s
Wall time: 1min 39s


In [21]:
df['text'].head(5)

5    Как стало известно агентству Ассошиэйтед Пресс...
6    В зале игровых автоматов в третьем ярусе подзе...
7    Япония приняла решение разморозить кредиты Рос...
8    Британцы отмечают сегодня скорбную дату - втор...
9    В понедельник директор департамента внешних св...
Name: text, dtype: object

In [22]:
text.shape

(494799, 821732)

In [23]:
X_train, X_test, y_train, y_test = train_test_split(text, df['topic'], test_size=0.20, random_state=RANDOM_STATE)

In [24]:
X_train.shape

(395839, 821732)

In [25]:
X_test.shape

(98960, 821732)

In [26]:
clf = tree.DecisionTreeClassifier()

In [27]:
%%time
clf.fit(X_train,y_train)

CPU times: total: 1h 38min 41s
Wall time: 1h 38min 42s


DecisionTreeClassifier()

In [28]:
pred = clf.predict(X_test)
print('Качество модели по метрике F1', f1_score(y_test,pred,average='weighted'))

Качество модели по метрике F1 0.7906672993296563


### Изменим токенизатор

In [29]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [30]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\liz_o\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [31]:
stop_russian = stopwords.words('russian')

In [32]:
text_transformer = TfidfVectorizer(stop_words=stop_russian, 
                                   ngram_range=(1,1), 
                                   lowercase=True, 
                                   max_features=10000 )
text = text_transformer.fit_transform(df['text'])
X_train, X_test, y_train, y_test = train_test_split(text, df['topic'], test_size=0.2, random_state=RANDOM_STATE)

In [33]:
from sklearn.ensemble import GradientBoostingClassifier

In [34]:
X_train.shape

(395839, 10000)

In [35]:
%%time
clf2 = GradientBoostingClassifier()
clf2.fit(X_train,y_train)

CPU times: total: 3h 28min 27s
Wall time: 3h 29min 53s


GradientBoostingClassifier()

In [36]:
print('TRAIN - Качество модели по метрике F1 (токенизатор настроен)',
      f1_score(y_train,clf2.predict(X_train),average='weighted'))

TRAIN - Качество модели по метрике F1 (токенизатор настроен) 0.8560479482126485


In [37]:
%%time
pred2 = clf2.predict(X_test)
print('TEST - Качество модели по метрике F1 (токенизатор настроен)', f1_score(y_test,pred2,average='weighted'))

TEST - Качество модели по метрике F1 (токенизатор настроен) 0.8540777039276538
CPU times: total: 2.08 s
Wall time: 2.07 s


In [38]:
df['text'] = df['text'].str.replace('/','')

### приведем слова к первой форме

In [39]:
def remove_trash(list): 
    pattern = r'[^А-Яа-я]+'
    try:
      list = [re.sub(pattern, ' ', i) for i in list] 
#       print (list)
    except Exception as e:
      print(e)
    return list

In [40]:
df['text_clean'] = remove_trash(df['text'])

In [41]:
df[['text','text_clean']].head()

,text,text_clean
5,Как стало известно агентству Ассошиэйтед Пресс...,Как стало известно агентству Ассошиэйтед Пресс...
6,В зале игровых автоматов в третьем ярусе подзе...,В зале игровых автоматов в третьем ярусе подзе...
7,Япония приняла решение разморозить кредиты Рос...,Япония приняла решение разморозить кредиты Рос...
8,Британцы отмечают сегодня скорбную дату - втор...,Британцы отмечают сегодня скорбную дату вторую...
9,В понедельник директор департамента внешних св...,В понедельник директор департамента внешних св...


In [42]:
df.isna().sum()

url               0
title             0
text              0
topic             0
tags          15328
date              0
text_clean        0
dtype: int64

In [43]:
#df = df[~df.text_clean.isna()].copy()

In [44]:
df.head()

,url,title,text,topic,tags,date,text_clean
5,https://lenta.ru/news/1999/08/31/stancia_mir/,"Космонавты сомневаются в надежности ""Мира""",Как стало известно агентству Ассошиэйтед Пресс...,Россия,Все,1999/08/31,Как стало известно агентству Ассошиэйтед Пресс...
6,https://lenta.ru/news/1999/08/31/vzriv/,Взрыв в центре Москвы: пострадало 30 человек,В зале игровых автоматов в третьем ярусе подзе...,Россия,Все,1999/08/31,В зале игровых автоматов в третьем ярусе подзе...
7,https://lenta.ru/news/1999/08/31/credit_japs/,Япония кредитует Россию на полтора миллиарда д...,Япония приняла решение разморозить кредиты Рос...,Россия,Все,1999/08/31,Япония приняла решение разморозить кредиты Рос...
8,https://lenta.ru/news/1999/08/31/diana/,Британцы отмечают двухлетие смерти Дианы,Британцы отмечают сегодня скорбную дату - втор...,Мир,Все,1999/08/31,Британцы отмечают сегодня скорбную дату вторую...
9,https://lenta.ru/news/1999/08/31/mvf/,Отмытые через Bank of NY деньги не имели отнош...,В понедельник директор департамента внешних св...,Россия,Все,1999/08/31,В понедельник директор департамента внешних св...


In [45]:
%%time
text_transformer = TfidfVectorizer(stop_words=stop_russian, 
                                   ngram_range=(1,1), 
                                   lowercase=True, 
                                   max_features=10000 )
text = text_transformer.fit_transform(df['text_clean'])
X_train, X_test, y_train, y_test = train_test_split(text, df['topic'], test_size=0.2, random_state=RANDOM_STATE)

CPU times: total: 1min 16s
Wall time: 1min 17s


In [46]:
%%time
clf3 = tree.DecisionTreeClassifier(max_depth=3)
clf3.fit(X_train,y_train)
pred3 = clf3.predict(X_test)
print('Качество модели по метрике F1 (токенизатор настроен и убран мусор)', f1_score(y_test,pred3,average='weighted'))

Качество модели по метрике F1 (токенизатор настроен и убран мусор) 0.32884454455872786
CPU times: total: 19.1 s
Wall time: 19.2 s


### попробуем лемматизацию

In [47]:
morph = pymorphy2.MorphAnalyzer()

In [48]:
# функция лемматизации
def lemmatize(row):
    t = []
    text = row['text_clean']
    for word in text.split():
        if len(word)<=1:
            continue
        p = morph.parse(word)[0]
        t.append(p.normal_form)
    return " ".join(t)

In [49]:
df.text

5         Как стало известно агентству Ассошиэйтед Пресс...
6         В зале игровых автоматов в третьем ярусе подзе...
7         Япония приняла решение разморозить кредиты Рос...
8         Британцы отмечают сегодня скорбную дату - втор...
9         В понедельник директор департамента внешних св...
                                ...                        
739168    Президент России Владимир Путин, выступая на з...
739169    Президент России Владимир Путин организовал дл...
739171    Протесты движения «желтых жилетов» в Париже сн...
739174    Швейцарский горнолыжник Марк Гизин неудачно пр...
739175    Президент России Владимир Путин рассказал, что...
Name: text, Length: 494799, dtype: object

In [50]:
t = []
text = 'Минфин предлагает увеличить размер штрафа'
for word in text.split():
    if len(word)<=3:
        continue
    p = morph.parse(word)[0]
    t.append(p.normal_form)

In [51]:
" ".join(t)

'минфин предлагать увеличить размер штраф'

In [52]:
%%time
df['text_clean_normal'] = df.apply(lemmatize,axis=1)

CPU times: total: 4h 22min 35s
Wall time: 4h 22min 40s


In [53]:
# Запись в файл объекта
with open('my_newfile.pkl','wb') as f:
    pickle.dump(df['text'],f)

In [54]:
#Чтение объекта в память
with open ('my_newfile.pkl','rb') as f:
    data_df = pickle.load(f)

In [55]:
# пример записи объекта в файл
with open('df_norma.pkl','wb') as f:
    pickle.dump(df,f)

In [56]:
# пример чтения объекта из файла в оперативную память
with open('df_norma.pkl','rb') as f:
    df = pickle.load(f)

In [57]:
df.head()

,url,title,text,topic,tags,date,text_clean,text_clean_normal
5,https://lenta.ru/news/1999/08/31/stancia_mir/,"Космонавты сомневаются в надежности ""Мира""",Как стало известно агентству Ассошиэйтед Пресс...,Россия,Все,1999/08/31,Как стало известно агентству Ассошиэйтед Пресс...,как стать известно агентство ассошиэйтед пресс...
6,https://lenta.ru/news/1999/08/31/vzriv/,Взрыв в центре Москвы: пострадало 30 человек,В зале игровых автоматов в третьем ярусе подзе...,Россия,Все,1999/08/31,В зале игровых автоматов в третьем ярусе подзе...,зал игровой автомат третий ярус подземный комп...
7,https://lenta.ru/news/1999/08/31/credit_japs/,Япония кредитует Россию на полтора миллиарда д...,Япония приняла решение разморозить кредиты Рос...,Россия,Все,1999/08/31,Япония приняла решение разморозить кредиты Рос...,япония принять решение разморозить кредит росс...
8,https://lenta.ru/news/1999/08/31/diana/,Британцы отмечают двухлетие смерти Дианы,Британцы отмечают сегодня скорбную дату - втор...,Мир,Все,1999/08/31,Британцы отмечают сегодня скорбную дату вторую...,британец отмечать сегодня скорбный дата вторую...
9,https://lenta.ru/news/1999/08/31/mvf/,Отмытые через Bank of NY деньги не имели отнош...,В понедельник директор департамента внешних св...,Россия,Все,1999/08/31,В понедельник директор департамента внешних св...,понедельник директор департамент внешний связь...


In [58]:
df[['text','text_clean_normal']].tail(10)

,text,text_clean_normal
739161,Одиннадцать человек скончались после приема пи...,одиннадцать человек скончаться после приём пищ...
739162,Россия не настроена серьезно на сохранение Дог...,россия не настроить серьёзно на сохранение дог...
739164,Федеральный суд в техасском городе Форт-Уорт о...,федеральный суд техасский город форт уорт объя...
739166,Россия призвала Китай принять участие в перего...,россия призвать китай принять участие перегово...
739167,Главный тренер «Манчестер Юнайтед» Жозе Моурин...,главный тренер манчестер юнайтед жозе моуринья...
739168,"Президент России Владимир Путин, выступая на з...",президент россия владимир путин выступать на з...
739169,Президент России Владимир Путин организовал дл...,президент россия владимир путин организовать д...
739171,Протесты движения «желтых жилетов» в Париже сн...,протест движение жёлтый жилет париж снова прив...
739174,Швейцарский горнолыжник Марк Гизин неудачно пр...,швейцарский горнолыжник марк гизина неудачно п...
739175,"Президент России Владимир Путин рассказал, что...",президент россия владимир путин рассказать что...


### качество модели после лемматизации

In [61]:
text_transformer = TfidfVectorizer(stop_words=stop_russian, 
                                   ngram_range=(1,1), 
                                   lowercase=True, 
                                   max_features=10000)
text_norm = text_transformer.fit_transform(df['text_clean_normal'])
X_train, X_test, y_train, y_test = train_test_split(text_norm, df['topic'], test_size=0.20, random_state=RANDOM_STATE)

In [ ]:
# text_transformer.transform(['Минфин предлагает увеличить размер'])

In [62]:
clf_norm = tree.DecisionTreeClassifier()
clf_norm.fit(X_train,y_train)
pred_norm = clf_norm.predict(X_test)

In [63]:
# clf_norm.predict(text_transformer.transform(['Минфин предлагает увеличить размер']))

In [64]:
print('Качество модели по метрике F1 после лемматизации', f1_score(y_test,pred_norm,average='weighted'))

Качество модели по метрике F1 после лемматизации 0.814645292785442


## Сравниваем вектора текстов

In [66]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


textA = text_transformer.transform(['бывший тренер сборная россия владимир плющев']   )                              

test = cosine_similarity(text_norm, textA)

In [67]:
testdf = pd.DataFrame(test)
testdf.columns=['vector']

In [68]:
testdf.sort_values(by='vector',ascending=False).head(5)

,vector
79742,0.573275
316961,0.558445
46511,0.515492
45830,0.511427
288703,0.502407


In [70]:
df[df.index==79742]['text_clean_normal'].values

array(['тольятти убить главный редактор газета тольяттинский обозрение алексей сидоров сообщать риа новость по информация гувд самарский область преступление совершенно четверг по московский время двое неизвестный напасть на сидоров около подъезд он дом нанести он два ножевой ранение спина как отмечаться сидоров возглавить газета около полтора год назад после убийство предыдущий главное редактор тольяттинский обозрение валерий иванов убийца иванов до сей пора не найти по факт преступление возбудить уголовный дело гувд основной мотив убийство связывать профессиональный деятельность журналист'],
      dtype=object)